# PyTorch: Aprendiendo Fashion-MNIST

## Refs.

* https://pytorch.org/tutorials/beginner/basics/optimization_tutorial.html

* https://github.com/zalandoresearch/fashion-mnist

* https://github.com/pranay414/Fashion-MNIST-Pytorch/blob/master/fashion_mnist.ipynb

In [1]:
# 1.1)
import os
import pickle
import datetime
from collections import defaultdict

In [2]:
# 1.2)
import matplotlib.pyplot as plt
from matplotlib import cm
import numpy as np
import scipy as sp
import scipy.linalg as linalg
import sklearn as skl
import pandas as pd
#import dill
import json

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
# 1.3)
import torch
import torch.optim as optim
from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader, Subset, random_split
from torchvision import datasets
from torchvision import transforms
from torchvision.io import read_image
from torchvision.transforms import ToTensor, Lambda, Compose
#from torchviz import make_dot

ModuleNotFoundError: No module named 'torch'

In [ ]:
# 1.4)
import google.colab
from google.colab import files

## **Ejercicio 2)**

Bajando y Jugando con el dataset **Fashion-MNIST**.

**1)** Baje y transforme (i.e. normalize los valores al rango [0,1]) los conjuntos de entrenamiento y testeo de FashionMNIST.

**2)** Explore algunos ejemplos de estos conjuntos. Que formato poseen?

**3)** Visitando la página web de FashionMNIST, cree un diccionario de Python `Dict()` asociando cada categoría a un nombre adecuado de la misma.

**4)** Grafique un mosaico de 3x3 imagenes de FashionMNIST, cada una titulada con su respectiva clasificación

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),  # Convertir las imágenes a tensores
])                          # Esto ya normaliza los valores entre 0 y 1


train_dataset = datasets.FashionMNIST(
    root="./data",       # Directorio donde se almacenará el dataset
    train=True,          
    download=True,       
    transform=transform  # Aplicar transformaciones
)

test_dataset = datasets.FashionMNIST(
    root="./data",
    train=False,         # False para descargar el conjunto de prueba
    download=True,
    transform=transform
)

100%|██████████| 26.4M/26.4M [00:03<00:00, 8.54MB/s]


Extracting MNIST_data/FashionMNIST/raw/train-images-idx3-ubyte.gz to MNIST_data/FashionMNIST/raw



100%|██████████| 29.5k/29.5k [00:00<00:00, 134kB/s]


Extracting MNIST_data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to MNIST_data/FashionMNIST/raw



100%|██████████| 4.42M/4.42M [00:01<00:00, 2.56MB/s]


Extracting MNIST_data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to MNIST_data/FashionMNIST/raw



100%|██████████| 5.15k/5.15k [00:00<00:00, 14.8MB/s]

Extracting MNIST_data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST_data/FashionMNIST/raw



In [ ]:
fashion_mnist_labels = {
    0: "T-shirt/top",
    1: "Trouser",
    2: "Pullover",
    3: "Dress",
    4: "Coat",
    5: "Sandal",
    6: "Shirt",
    7: "Sneaker",
    8: "Bag",
    9: "Ankle boot"
}


fig, axes = plt.subplots(3, 3, figsize=(8, 8))
for i, ax in enumerate(axes.flatten()):
    image, label = train_dataset[i]  
    image = image.squeeze()  # Eliminar la dimensión extra (El canal de grises)
    ax.imshow(image, cmap="gray")  
    ax.set_title(fashion_mnist_labels[label])  
    ax.axis("off")  

plt.tight_layout()
plt.show()

## Ejercicio 3)

Creando un `DataLoader` para alimentar el modelo con batchs (lotes) de entrenamiento.

**1)** Cree los `DataLoader`s para cada conjunto. Defínalos con un `batch_size` de 100 y con el flag `shuffle` seteado a `True`.

**2)** Use uno de los `DataLoader`s creados anteriormente para explorar algunos elementos del conjunto.

Notar que, el iterador devuelve el batch en un par `(image,label)`.

El objeto `images` es un tensor de dimensiones `(100,1,28,28)`.
El 100 es el tamaño del batch.
El 1 porque hay un solo canal (en este caso, un canal de escala de grises, pero podría haber varios, p. ej. uno por cada color de {Red, Green Blue} en caso que fuesen imagenes a color).
Luego, 28 y 28 porque cada imagen del dataset es de 28 x 28 píxeles.

El objeto `labels` es un tensor de dimensiones `(100,)`.
La $i$-ésima entrada `labels[i]` de `labels` es un número en $\{0,1,...,9\}$ indicando la categoría a la que pertenece la $i$-ésima imagen en el batch, guardada en `images[i]`.

In [ ]:
train_set, valid_set = random_split(train_dataset, [50000, 10000])  # Dividir en train y validación

train_loader = DataLoader(dataset=train_set, batch_size=100, shuffle=True)
valid_loader = DataLoader(dataset=valid_set, batch_size=100, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=100, shuffle=True)

# Verificar que se haya cargado correctamente
for images, labels in train_loader:
    print(f"Lote de imágenes: {images.size()}, Lote de etiquetas: {labels.size()}")
    break

## Ejercicio 4)

Defina una red neuronal de 4 capas, una de entrada, dos ocultas de $n_1=128$ y $n_2=64$ neuronas, respectivamente, y una de salida de 10 neuronas.

En las capas intermedias utilice neuronas tipo ReLU y agregueles un *dropout* de p=0.2.
En la capa de salida no utilice funciones de activación ni dropout.

Las capas sucesivas tienen que estar totalmente conectadas entre si.

In [ ]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, output_size):
        super(NeuralNet, self).__init__()

        # Definir las capas de la red
        self.model = nn.Sequential(
            nn.Linear(input_size, 128),  # Capa de entrada a la primera capa oculta
            nn.ReLU(),                  # Activación ReLU
            nn.Dropout(p=0.2),          # Dropout con probabilidad 0.2
            nn.Linear(128, 64),         # Primera capa oculta a la segunda
            nn.ReLU(),                  # Activación ReLU
            nn.Dropout(p=0.2),          # Dropout con probabilidad 0.2
            nn.Linear(64, output_size)  # Segunda capa oculta a la salida
            # Sin función de activación ni dropout en la capa de salida
        )

    def forward(self, x):
        return self.model(x)


input_size = 28 * 28  
output_size = 10     

model = NeuralNet(input_size, output_size)

## Ejercicio 5)

Entrenamos el modelo

**1)** Implemente, en una función, un loop de entrenamiento que recorra los batchs (lotes).

**2)** Implemente, en una función, un loop de validación que recorra los batchs.

**3)** Inicialize dos `DataLoader`s llamados `train_loader` y `valid_loader` a partir del `train_set` (conjunto de entranmiento) y del `valid_set` (conjunto de validación) de Fashion-MNIST, respectivamente, y que usen batchs de 100 ejemplos.

**4)** Cree una función de pérdida usando la **Cross Entropy Loss**.

**IMPORTANTE:** Notar que la **Cross Entropy Loss** aplica automáticamente una `log_softmax`.

**5)** Cree un optimizador que utilice el método de **Stochastic Gradient Descent** con un learning rate igual a $10^{-3}$.

**6)** Cree una instancia del modelo.

**7)** Especifique en que dispositivo (`device`) va a trabajar: en una **CPU** o en una **GPU**.

**8)** Implemente un loop de entrenamiento y validación que trabaje con el `train_loader` y el `valid_loader`, respectivamente, usando un numero arbitrario de épocas.
Este loop debe guardar en cuatro listas los valores de los promedios del **Cross Entropy Loss** y las fracciones de clasificaciones correctas o **precisión** (accuracy) sobre el conjunto de **entrenamiento** y el de **validación**, respectivamente.

**IMPORTANTE:** No olvide copiar los batchs al dispositivo de trabajo.

**9)** Entrene y valide el modelo.

**10)** Use las listas del inciso anterior para graficar en función de las épocas la **Cross Entropy Loss** de **entrenamiento** y de **validación**.
Realize un gráfico análogo pero con la **precisión**.
Discuta y comente, cual es el número óptimo de épocas de entrenamiento?

**11)** Repita los experimentos variando hiperparámetros. Por ejemplo:

- El learning-rate.
- El optimizador (ej. puede usar ADAM).
- El valor de dropout.
- El número de neuronas en las capas intermedias.
- El número de épocas de entrenamiento.
- El tamaño de los lotes.

Discuta los resultados.

In [ ]:
def train_loop(model, train_loader, criterion, optimizer, device):
    model.train()  # Poner la red en modo de entrenamiento
    total_loss = 0
    correct = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)  
        data = data.view(data.size(0), -1)  # Aplanar las imágenes (28x28 -> 784)

        optimizer.zero_grad()  # Limpiar gradientes previos
        output = model(data)  # Paso hacia adelante
        loss = criterion(output, target)  # Calcular la pérdida
        loss.backward()  # Paso hacia atrás
        optimizer.step()  # Actualizar los parámetros

        total_loss += loss.item()
        pred = output.argmax(dim=1, keepdim=True)  # Obtener predicciones
        correct += pred.eq(target.view_as(pred)).sum().item()

    avg_loss = total_loss / len(train_loader)
    accuracy = 100. * correct / len(train_loader.dataset)
    return avg_loss, accuracy


In [ ]:
def valid_loop(model, valid_loader, criterion, device):
    model.eval()  # Poner la red en modo de evaluación
    total_loss = 0
    correct = 0

    with torch.no_grad():  # Desactivar el cálculo de gradientes
        for batch_idx, (data, target) in enumerate(valid_loader):
            data, target = data.to(device), target.to(device)  
            data = data.view(data.size(0), -1)  

            output = model(data)  # Paso hacia adelante
            loss = criterion(output, target)  # Calcular la pérdida

            total_loss += loss.item()
            pred = output.argmax(dim=1, keepdim=True)  # Obtener predicciones
            correct += pred.eq(target.view_as(pred)).sum().item()

    avg_loss = total_loss / len(valid_loader)
    accuracy = 100. * correct / len(valid_loader.dataset)

    return avg_loss, accuracy


In [ ]:
# Definir la función de pérdida
criterion = nn.CrossEntropyLoss()

In [ ]:
learning_rate = 1e-3
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(f"Modelo trabajando en: {device}")

In [ ]:
num_epochs = 100
train_losses, train_accuracies = [], []
valid_losses, valid_accuracies = [], []

for epoch in range(1, num_epochs + 1):
    print(f"\nÉpoca {epoch}/{num_epochs}")

    # Entrenamiento
    train_loss, train_accuracy = train_loop(model, train_loader, criterion, optimizer, device)
    train_losses.append(train_loss)
    train_accuracies.append(train_accuracy)

    # Validación
    valid_loss, valid_accuracy = valid_loop(model, valid_loader, criterion, device)
    valid_losses.append(valid_loss)
    valid_accuracies.append(valid_accuracy)

    print(f"Resumen - Época {epoch}:")
    print(f"  Pérdida de entrenamiento: {train_loss:.4f}")
    print(f"  Pérdida de validación: {valid_loss:.4f}, Precisión de validación: {valid_accuracy:.2f}%")

print("\nEntrenamiento finalizado.")

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(15, 5))

# Gráfico de pérdidas (Train vs Validation)
ax[0].plot(train_losses, label='Pérdida de Entrenamiento', color='blue', linestyle='-', marker='o')
ax[0].plot(valid_losses, label='Pérdida de Validación', color='orange', linestyle='--', marker='s')
ax[0].set_title('Pérdidas durante el Entrenamiento', fontsize=14)
ax[0].set_xlabel('Épocas', fontsize=12)
ax[0].set_ylabel('Pérdida', fontsize=12)
ax[0].legend(fontsize=10)
ax[0].grid(True, linestyle='--', alpha=0.7)

# Gráfico de precisiones (Train vs Validation)
ax[1].plot(train_accuracies, label='Precisión de Entrenamiento', color='green', linestyle='-', marker='o')
ax[1].plot(valid_accuracies, label='Precisión de Validación', color='red', linestyle='--', marker='s')

ax[1].set_title('Precisión durante el Entrenamiento', fontsize=14)
ax[1].set_xlabel('Épocas', fontsize=12)
ax[1].set_ylabel('Precisión (%)', fontsize=12)
ax[1].legend(fontsize=10)
ax[1].grid(True, linestyle='--', alpha=0.7)

plt.tight_layout()

plt.show()